In [23]:
model_checkpoint = 'google/mt5-small'

In [1]:
!pip install transformers datasets seacrowd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 19.8 MB/s eta 0:00:00
   ━

In [25]:
from datasets import load_dataset
dset = load_dataset("SEACrowd/liputan6", trust_remote_code=True)

# Use subsets of the dataset
train_data = dset["train"].select(range(2000))
val_data = dset["validation"].select(range(200))
test_data = dset["test"].select(range(200))

In [26]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, DataCollatorWithPadding

tokenizer_ft_mt5 = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    # Tokenisasi teks input
    model_inputs = tokenizer_ft_mt5(
        examples['document'],
        max_length=512,
        truncation=True,
        padding='max_length'  # Add padding to max_length
    )
    # Tokenisasi target
    with tokenizer_ft_mt5.as_target_tokenizer():
        labels = tokenizer_ft_mt5(
            examples['summary'],
            max_length=128,
            truncation=True,
            padding='max_length'  # Add padding to max_length
        )
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset_train = train_data.map(tokenize_function)
tokenized_dataset_val = train_data.map(tokenize_function)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [27]:
print(tokenized_dataset_train)
print(tokenized_dataset_val)

Dataset({
    features: ['document', 'id', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2000
})
Dataset({
    features: ['document', 'id', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2000
})


In [28]:
tokenized_dataset_train = tokenized_dataset_train.remove_columns(["document", "id", "summary"])
tokenized_dataset_val = tokenized_dataset_val.remove_columns(["document", "id", "summary"])

In [29]:
print(tokenized_dataset_train)
print(tokenized_dataset_val)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2000
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2000
})


In [30]:
# Parameter
learning_rate = 5e-5  # Laju pembelajaran
train_batch_size = 6    # Ukuran batch untuk pelatihan
epochs = 10              # Jumlah epoch
weight_decay = 0.001      # Pengurangan bobot

# Direktori untuk menyimpan hasil
output_directory = f'./results_mt5_{learning_rate}_{train_batch_size}_{epochs}'

In [31]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model_ft_mt5 = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Use DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer_ft_mt5)

training_args = TrainingArguments(
    output_dir = output_directory,          # direktori untuk menyimpan model
    evaluation_strategy="epoch",     # evaluasi setiap epoch
    learning_rate=learning_rate,
    per_device_train_batch_size=train_batch_size,   # ukuran batch untuk pelatihan
    num_train_epochs=epochs,              # jumlah epoch
    weight_decay=weight_decay,               # pengurangan bobot
)


trainer = Trainer(
    model=model_ft_mt5,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    data_collator=data_collator,  # Use the data collator
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [32]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: nadhiefathallahi (nadhiefathallahi-universitas-pendidikan-indonesia). Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,10.244788
2,18.215400,4.345018
3,5.466500,3.056386
4,5.466500,2.187997
5,3.298400,0.971900
6,1.756600,0.867901
7,1.756600,0.844834
8,1.275700,0.825849
9,1.176800,0.811749
10,1.176800,0.808245


TrainOutput(global_step=3340, training_loss=4.786505565529098, metrics={'train_runtime': 3275.4281, 'train_samples_per_second': 6.106, 'train_steps_per_second': 1.02, 'total_flos': 1.05749938176e+16, 'train_loss': 4.786505565529098, 'epoch': 10.0})

In [72]:
new_article = dset["test"][1]["document"]
new_summary = dset["test"][1]["summary"]
print(new_article)

Liputan6 . com , Bandar Lampung : Sebanyak 51 anak di bawah umur lima tahun terserang busung lapar atau marasmus karena kekurangan gizi di Kota Madya Bandar Lampung . Lima di antaranya tewas . Data tersebut diungkapkan Kepala Dinas Kesehatan Kota Bandar Lampung M . Sudarman , baru-baru ini . Menurut Sudarman , Dinas Kesehatan Bandar Lampung mencatat sekitar 51 anak terserang busung lapar yang tersebar di beberapa kecamatan , selama periode 1999 sampai 2001 . Kebanyakan anak penderita busung tersebut berasal dari keluarga yang hidup di bawah garis kemiskinan . Selain kekurangan gizi , komplikasi radang paru-paru juga menjadi satu faktor penyebab kematian anak penderita busung lapar tersebut . Data Dinas Kesehatan menunjukkan pada 1999 , ditemukan 41 anak terserang penyakit busung lapar . Sebagian besar penderita berdomisili di kampung miskin Umbul Kunci . Jumlah penderita busung lapar menurun pada 2000 , yakni hanya sembilan anak . Sedangkan September 2001 , seorang anak meninggal karen

In [73]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_result_mt5 = AutoTokenizer.from_pretrained(model_checkpoint)
model_result_mt5 = AutoModelForSeq2SeqLM.from_pretrained('/content/results_mt5_5e-05_6_10/checkpoint-3340')

# Teks input
input_text = new_article

# prompt
prompt = f"""
summary artikel dibawah ini:
Article: {new_article}
Summary:"""

# Tokenisasi
input_teks = prompt + input_text
input_ids = tokenizer_result_mt5.encode(input_teks, return_tensors='pt')

# Menghasilkan ringkasan
output_ids = model_result_mt5.generate(input_ids, max_length=70)
output_text = tokenizer_result_mt5.decode(output_ids[0], skip_special_tokens=True)

# print("Referensi: ", referensi_ringkasan)
print("Ringkasan Hasil Fine Tune:", output_text)

Ringkasan Hasil Fine Tune: <extra_id_0> di Kota Madya Bandar Lampung . Lima diantaranya tewas .


In [59]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
# Calculate BLEU score for this sentence
bleu_score = sentence_bleu([new_summary], output_text)

# Print the results
print(f"Input: {new_article}")
print(f"Predicted: {output_text}")
print(f"Reference: {new_summary}")
print(f"BLEU Score: {bleu_score:.4f}")
print("-" * 50)

Input: Liputan6 . com , Bandar Lampung : Sebanyak 51 anak di bawah umur lima tahun terserang busung lapar atau marasmus karena kekurangan gizi di Kota Madya Bandar Lampung . Lima di antaranya tewas . Data tersebut diungkapkan Kepala Dinas Kesehatan Kota Bandar Lampung M . Sudarman , baru-baru ini . Menurut Sudarman , Dinas Kesehatan Bandar Lampung mencatat sekitar 51 anak terserang busung lapar yang tersebar di beberapa kecamatan , selama periode 1999 sampai 2001 . Kebanyakan anak penderita busung tersebut berasal dari keluarga yang hidup di bawah garis kemiskinan . Selain kekurangan gizi , komplikasi radang paru-paru juga menjadi satu faktor penyebab kematian anak penderita busung lapar tersebut . Data Dinas Kesehatan menunjukkan pada 1999 , ditemukan 41 anak terserang penyakit busung lapar . Sebagian besar penderita berdomisili di kampung miskin Umbul Kunci . Jumlah penderita busung lapar menurun pada 2000 , yakni hanya sembilan anak . Sedangkan September 2001 , seorang anak meningga

In [74]:
from nltk.translate.bleu_score import sentence_bleu
# List untuk menyimpan skor BLEU
bleu_scores = []

# Pastikan `dset["test"]` memiliki 10 data
for i in range(10):
    # Teks input dari dataset
    new_article = dset["test"][i]["document"]  # Pastikan `document` adalah kunci yang benar

    # Prompt untuk model
    prompt = f"""
    summary artikel dibawah ini:
    Article: {new_article}
    Summary:"""

    # Tokenisasi
    input_teks = prompt
    input_ids = tokenizer_result_mt5.encode(input_teks, return_tensors='pt')

    # Menghasilkan ringkasan
    output_ids = model_result_mt5.generate(input_ids, max_length=70)
    output_text = tokenizer_result_mt5.decode(output_ids[0], skip_special_tokens=True)

    # Hitung BLEU score
    bleu_score = sentence_bleu([dset["test"][i]["summary"]], output_text)  # `hypothesis` diubah menjadi list of words
    bleu_scores.append(bleu_score)

    # Cetak hasil setiap iterasi
    print(f"Input: {new_article}")
    print(f"Predicted: {output_text}")
    print(f"Reference: {dset['test'][i]['summary']}")
    print(f"BLEU Score: {bleu_score:.4f}")
    print("-" * 50)

# Hitung rata-rata BLEU score
average_bleu_score = sum(bleu_scores) / len(bleu_scores)

# Cetak rata-rata BLEU score
print(f"Average BLEU Score for 10 samples: {average_bleu_score:.4f}")


Input: Liputan6 . com , Bangka : Kapal patroli Angkatan Laut Republik Indonesia , Belinyu , baru-baru ini , menangkap tiga kapal nelayan berbendera Thailand , yakni KM Binatama , KM Sumber Jaya II , dan KM Mataram di Perairan Belitung Utara . Ketiga kapal itu ditangkap karena melanggar zona ekonomi ekslusif Indonesia . Saat ini , kapal-kapal itu diamankan di Pos Lanal Pelabuhan Pangkalan Balam , Bangka-Belitung . Menurut Komandan Pangkalan TNI AL Bangka Letnan Kolonel Laut Fredy Egam , selain menangkap tiga kapal , ALRI juga memeriksa 43 anak buah kapal . Mereka disergap saat sedang mengangkat jaring pukat harimau di Perairan Belitung Utara . Dari jumlah itu , hanya enam orang yang dijadikan tersangka , yakni tiga nahkoda dan tiga kepala kamar mesin kapal . Sedangkan ABK yang lain akan dideportasi ke negara asalnya . Meski berhasil menahan enam tersangka , TNI AL gagal mengamankan ikan tangkapan nelayan Thailand tersebut . Sebab , sebelum patroli datang , mereka telah memindahkan puluh